# RGA Tool 

Things to Keep in Mind:

1. We should make different models for properties with different numbers of bedrooms.
2. There are many details about a property that we know before it's listed. We should use these in our model.(refer the code       to see it)
3. We need to adjust our approach for different sized properties.
4. Our model seems too accurate. We shouldn't trust it fully, especially since the three  factors we used don't seem that 
    important on their own also based on their shap value which is not very high.
5. We used cross-validation to test our model.

Validation Steps:

1. High chance of data leakage in the current model.
2. Possible overfitting as the Lag period and availability seem interdependent.
3. Implement a separate hold-out test set to verify model performance independently of cross-validation.
4. Maybe use simpler models (e.g., logistic regression) to benchmark performance.
5. Run couple of more classifcation model to make sure the outcome is reliable. 

In [2]:
import pandas as pd
import numpy as np
from datetime import date

import platform
import pyodbc

from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text

import matplotlib.pyplot as plt
import dtale
import sweetviz
# import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split
from scipy.interpolate import UnivariateSpline

pd.set_option('display.max.columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [1]:
import os

# Change the current working directory
os.chdir('C:/Users/DeepankarSrigyan/OneDrive - Sykes Cottages Ltd/Desktop/Project/RGA Tools/')

In [3]:
def fetch_SQLserver_data(sql_code,conn_string):
    """
    Uses pyodbc.connect and pandas.read_sql to return query results as dataframe.
    :param sql_code: string, query to be submitted
    :param conn_string: string, full connection details as string
    """
    connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string})
    engine = create_engine(connection_url)
    with engine.connect() as conn:
        df = pd.read_sql(sql=text(sql_code), con=conn)
        conn.commit()
    print('Data Imported')
    return df

if platform.system() == 'Windows':
    conn_CDM = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=CDM;trusted_connection=Yes'
    conn_l0 = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Sykes_DW_Level0_Staging;trusted_connection=Yes'
    conn_l2 = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Sykes_DW_Level2_ADM;trusted_connection=Yes'
    conn_BOX = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Box;trusted_connection=Yes'

    print ("Connection Completed")

Connection Completed


In [4]:
PropertyImage= pd.read_excel("PropertyWithImage.xlsx").reset_index(drop=True)
PropertyImage

,PropertyID,PropertyVersionStartDateLocal,PropertyReference,PropertyReferenceType,ParkReference,ParkReferenceType,PropertyLeadID,BrandID,OwnerID,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,Latitude,Longitude,ParkName,PricingArea,PropertyName,PropertyStatus,PropertyType,Rating,RatingType,Sleeps,SykesTicks,isHoneyPot,isTestProperty,Northings,Eastings,LocationLongName,LocationType,County,Country,Location,WorkingRegion,PropertyLiveDate_x,AvailableFromDate,AvailableToDate,DateToldWithdrawingLocal,WithdrawalReason,WithdrawalWhatReason,WithdrawalWhyReason,StopSell,CommissionRate,RevenueManagementOption,ReinstatementDate,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,hasSpecialOffers,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual,PropertyLiveDate_y,PicsAtLive,ChurnFlag,difference_days,difference_years,Churn_Year_1,Churn_Year_2,Churn_Year_1&2
0,203860,2024-03-18 22:50:58,1113426,SykesPropertyID,SykesPark:0,SykesParkID,232532.0,1,470883.0,True,1,1,1,Friday,52.292820,-2.336594,NaN,Worcestershire,Tamina,Current,Hut/Pod/Yurt,4,Ticks 1 to 5,6.0,4.0,False,False,265500.0,377500.0,"Structon's Heath, Worcestershire",Other settlement,Worcestershire,England,Structon's Heath,Heart of England,2022-08-18,2022-09-01,NaT,NaT,Unknown,Unknown,Unknown,False,15.0,Super Income Maximisation,NaT,True,False,False,False,True,True,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,True,False,False,True,False,False,True,False,False,False,True,True,True,False,False,True,True,True,False,False,True,False,False,2022-08-18,20,0,NaN,NaN,0,0,0
1,203868,2024-02-17 03:22:09,1113505,SykesPropertyID,SykesPark:0,SykesParkID,232587.0,402,473298.0,False,0,2,2,Saturday,54.328608,-2.749704,NaN,Cumbria West of M6,Joiners Nook,Lapsed,House terraced,4,Ticks 1 to 5,4.0,4.0,False,False,492500.0,351500.0,"Kendal, Cumbria",Town,Cumbria,England,Kendal,Cumbria & The Lake District,2022-11-11,2022-11-22,2024-02-16,2023-08-17,Selling,Selling,Not Impacted by Sykes,False,18.0,Super Income Maximisation,NaT,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,True,True,False,True,False,True,True,True,False,True,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,2022-11-11,23,1,279.0,1.0,1,0,1
2,203872,2023-05-03 22:59:20,1113524,SykesPropertyID,SykesPark:0,SykesParkID,232596.0,1,473277.0,True,1,1,2,Saturday,50.872446,-4.492889,NaN,Devon,8 The Park,Current,Lodge,3,Ticks 1 to 5,4.0,3.0,False,False,111500.0,225500.0,"Kilkhampton, Cornwall",Other settlement,Cornwall,England,Kilkhampton,Cornwall,2022-09-29,2022-09-29,NaT,NaT,Unknown,Unknown,Unknown,False,22.0,Super Income Maximisation,NaT,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,True,True,False,False,False,False,2022-09-29,14,0,NaN,NaN,0,0,0
3,203875,2023-05-10 02:52:29,1113531,SykesPropertyID,SykesPark:0,SykesParkID,232578.0,1,413900.0,True,2,1,1,Friday,51.284965,-2.967237,NaN,HP Somerset Coast,The Hideaway,Current,Annex,4,Ticks 1 to 5,2.0,4.0,True,False,153500.0,332500.0,"Wick, Somerset",Other settlement,Somerset,England,Wick,Somerset,2023-05-09,2023-05-09,NaT,NaT,Unknown,Unknown,Unknown,False,14.0,Super Income Maximisation,NaT,Tr

In [5]:
availability=pd.read_excel('availability_outcome.xlsx').reset_index(drop=True)
availability

,PropertyID,PropertyReference,PropertyLiveDate,DateToldWithdrawingLocal,Difference,churn1year,AvailableFromDate,AvailableToDate,Available_Days,Not_Available_Days,Total_Days,LagPeriod
0,223990,1128768,2023-02-28,NaT,0,NaN,2023-03-03,NaT,362,3,365,3
1,225256,1129878,2023-08-25,NaT,0,NaN,2023-08-25,NaT,365,0,365,0
2,221376,1126413,2023-02-10,2023-10-23,255,1.0,2023-03-31,2023-10-30,214,151,365,49
3,279394,1152055,2024-06-14,NaT,0,NaN,2024-06-24,NaT,355,10,365,10
4,210746,1119529,2022-12-17,2023-04-13,117,1.0,2022-12-18,2023-12-29,364,1,365,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8171,284049,1155772,2024-05-09,NaT,0,NaN,2024-05-10,NaT,364,1,365,1
8172,277099,1150231,2024-02-29,NaT,0,NaN,2024-03-01,NaT,364,1,365,1
8173,234957,1138337,2023-09-11,NaT,0,NaN,2023-09-11,NaT,365,0,365,0
8174,273044,1147631,2024-03-13,NaT,0,NaN,2024-03-13,NaT,365,0,365,0


In [6]:
image_availability=pd.merge(availability,PropertyImage,how='left',on='PropertyID').reset_index(drop=True)
image_availability

,PropertyID,PropertyReference_x,PropertyLiveDate,DateToldWithdrawingLocal_x,Difference,churn1year,AvailableFromDate_x,AvailableToDate_x,Available_Days,Not_Available_Days,Total_Days,LagPeriod,PropertyVersionStartDateLocal,PropertyReference_y,PropertyReferenceType,ParkReference,ParkReferenceType,PropertyLeadID,BrandID,OwnerID,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,Latitude,Longitude,ParkName,PricingArea,PropertyName,PropertyStatus,PropertyType,Rating,RatingType,Sleeps,SykesTicks,isHoneyPot,isTestProperty,Northings,Eastings,LocationLongName,LocationType,County,Country,Location,WorkingRegion,PropertyLiveDate_x,AvailableFromDate_y,AvailableToDate_y,DateToldWithdrawingLocal_y,WithdrawalReason,WithdrawalWhatReason,WithdrawalWhyReason,StopSell,CommissionRate,RevenueManagementOption,ReinstatementDate,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,hasSpecialOffers,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual,PropertyLiveDate_y,PicsAtLive,ChurnFlag,difference_days,difference_years,Churn_Year_1,Churn_Year_2,Churn_Year_1&2
0,223990,1128768,2023-02-28,NaT,0,NaN,2023-03-03,NaT,362,3,365,3,2023-03-01 03:09:39,1128768.0,SykesPropertyID,SykesPark:0,SykesParkID,247374.0,389.0,526337.0,False,0.0,1.0,2.0,Friday,53.262607,-4.095680,NaN,HP - Wales Anglesey Coastal,Seagull Cottage,Current,Cottage,4.0,Ticks 1 to 5,4.0,4.0,True,False,376500.0,260500.0,"Beaumaris, Isle of Anglesey",Town,Isle of Anglesey,Wales,Beaumaris,North Wales,2023-02-28,2023-03-03,NaT,NaT,Unknown,Unknown,Unknown,False,15.0,Super Income Maximisation,NaT,True,False,True,False,True,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,True,True,False,True,False,False,False,True,False,True,True,False,False,True,False,True,True,False,True,True,False,True,False,False,False,False,False,2023-02-28,31.0,0.0,NaN,NaN,0.0,0.0,0.0
1,225256,1129878,2023-08-25,NaT,0,NaN,2023-08-25,NaT,365,0,365,0,2023-09-07 23:18:50,1129878.0,SykesPropertyID,SykesPark:0,SykesParkID,248432.0,365.0,483204.0,True,2.0,1.0,1.0,Friday,51.633817,-2.070215,NaN,Wiltshire,Well Cottage,Current,Cottage,4.0,Ticks 1 to 5,2.0,4.0,False,False,192500.0,395500.0,"Crudwell, Wiltshire",Other settlement,Wiltshire,England,Crudwell,Cotswolds,2023-08-25,2023-08-25,NaT,NaT,Unknown,Unknown,Unknown,False,15.0,Super Income Maximisation,NaT,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,True,True,True,False,False,False,False,True,False,False,False,False,False,2023-08-25,17.0,0.0,NaN,NaN,0.0,0.0,0.0
2,221376,1126413,2023-02-10,2023-10-23,255,1.0,2023-03-31,2023-10-30,214,151,365,49,2023-10-31 03:28:08,1126413.0,SykesPropertyID,SykesPark:0,SykesParkID,245112.0,1.0,519509.0,False,0.0,1.0,3.0,Friday,50.182851,-5.349730,NaN,Cornwall,Parbola 21,Lapsed,Caravan,3.0,Ticks 1 to 5,6.0,3.0,False,False,36500.0,160500.0,"Wall, Cornwall",Other settlement,Cornwall,England,Wall,Cornwall,2023-02-10,2023-03-31,2023-10-30,2023-10-23,Terminated By Sykes,Terminated By Sykes,Unwilling to Commit,False,15.0,Super Income Maximisation,NaT,True,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,False,False,True,False,True,False,False,False,True,False,True,True,False,False,True,True,True,True,False,False,True,False,True,False,False,False,False,False,2023-02-10,25.0,1

## Columns Details

In [7]:
def get_column_info(df):
    from IPython.display import display
    import pandas as pd
    
    pd.set_option('display.max.rows', None)
    
    column_info = pd.DataFrame({
        'Column_Name': df.columns,
        'Missing_Values': df.isnull().sum().values,
        'Unique_Values': df.nunique().values
    })
    
    display(column_info)
    
    # Reset the display option back to default if needed
    pd.set_option('display.max.rows', 100)
    
    # return column_info

# # Use the function
get_column_info(image_availability)


,Column_Name,Missing_Values,Unique_Values
0,PropertyID,0,8176
1,PropertyReference_x,0,8176
2,PropertyLiveDate,0,564
3,DateToldWithdrawingLocal_x,6525,372
4,Difference,0,497
5,churn1year,6525,2
6,AvailableFromDate_x,0,684
7,AvailableToDate_x,6525,547
8,Available_Days,0,329
9,Not_Available_Days,0,331


In [8]:
base=image_availability[['PropertyID',
'PropertyReference_x',
'PropertyLiveDate',
'DateToldWithdrawingLocal_x',
'Difference',
'AvailableFromDate_x',
'AvailableToDate_x',
'Available_Days',
'Not_Available_Days',
'Total_Days',
'LagPeriod',
'AllowsPets',
'NumberOfPetsAllowed',
'Bathrooms',
'Bedrooms',
'ChangeOverDay',
'PricingArea',
'PropertyType',
'Rating',
'Sleeps',
'SykesTicks',
'isHoneyPot',
'isTestProperty',
'LocationType',
'County',
'Country',
'Location',
'WorkingRegion',
'CommissionRate',
'RevenueManagementOption',
'hasBroadband',
'hasDishwasher',
'hasEnclosedGarden',
'hasGames',
'hasGarden',
'hasHotTub',
'hasIndoorPool',
'hasLivestock',
'hasOffRoadParking',
'hasOpenFire',
'hasPoolTable',
'hasSauna',
'hasSpaOrGym',
'hasSpaJacuzzi',
'hasSwimmingPool',
'hasWashingMachine',
'isCoastal',
'hasCountryside',
'hasFishing',
'isIsolated',
'hasNationalPark',
'hasNationalTrust',
'isNearPub',
'isNearShop',
'isNearCanal',
'isNearCycling',
'isNearLake',
'isNearRailway',
'isNearRiver',
'isNearWalks',
'isNew',
'hasNoAnimals',
'hasShortBreaksAllowed',
'isChildFriendly',
'hasCotAvailable',
'isGroundFloor',
'hasLastMinuteBreakAllowed',
'isSmokeFree',
'isSmoking',
'hasCharacter',
'isFamily',
'isFarm',
'isInGroup',
'isLogCabin',
'isLuxury',
'isRomantic',
'isThatched',
'isUnusual',
'PicsAtLive',
'ChurnFlag',
'Churn_Year_1',
'Churn_Year_2',
'Churn_Year_1&2',
]].rename(columns=
          {'PropertyReference_x': 'PropertyReference',
           'DateToldWithdrawingLocal_x':'DateToldWithdrawingLocal',
           'AvailableFromDate_x':'AvailableFromDate',
          'AvailableToDate_x':'AvailableToDate'})
base

,PropertyID,PropertyReference,PropertyLiveDate,DateToldWithdrawingLocal,Difference,AvailableFromDate,AvailableToDate,Available_Days,Not_Available_Days,Total_Days,LagPeriod,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,PricingArea,PropertyType,Rating,Sleeps,SykesTicks,isHoneyPot,isTestProperty,LocationType,County,Country,Location,WorkingRegion,CommissionRate,RevenueManagementOption,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual,PicsAtLive,ChurnFlag,Churn_Year_1,Churn_Year_2,Churn_Year_1&2
0,223990,1128768,2023-02-28,NaT,0,2023-03-03,NaT,362,3,365,3,False,0.0,1.0,2.0,Friday,HP - Wales Anglesey Coastal,Cottage,4.0,4.0,4.0,True,False,Town,Isle of Anglesey,Wales,Beaumaris,North Wales,15.0,Super Income Maximisation,True,False,True,False,True,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,True,True,False,True,False,False,False,True,False,True,True,False,True,False,True,True,False,True,True,False,True,False,False,False,False,False,31.0,0.0,0.0,0.0,0.0
1,225256,1129878,2023-08-25,NaT,0,2023-08-25,NaT,365,0,365,0,True,2.0,1.0,1.0,Friday,Wiltshire,Cottage,4.0,2.0,4.0,False,False,Other settlement,Wiltshire,England,Crudwell,Cotswolds,15.0,Super Income Maximisation,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,True,True,True,True,False,False,False,False,True,False,False,False,False,False,17.0,0.0,0.0,0.0,0.0
2,221376,1126413,2023-02-10,2023-10-23,255,2023-03-31,2023-10-30,214,151,365,49,False,0.0,1.0,3.0,Friday,Cornwall,Caravan,3.0,6.0,3.0,False,False,Other settlement,Cornwall,England,Wall,Cornwall,15.0,Super Income Maximisation,True,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,False,False,True,False,True,False,False,False,True,False,True,True,False,True,True,True,True,False,False,True,False,True,False,False,False,False,False,25.0,1.0,1.0,0.0,1.0
3,279394,1152055,2024-06-14,NaT,0,2024-06-24,NaT,355,10,365,10,False,0.0,1.0,3.0,Friday,HP Dorset Coast,House semi-detached,4.0,5.0,4.0,True,False,Other settlement,Dorset,England,Herston,Dorset,18.0,Full Income Maximisation,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,True,True,True,False,True,False,False,False,True,True,True,True,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,24.0,0.0,0.0,0.0,0.0
4,210746,1119529,2022-12-17,2023-04-13,117,2022-12-18,2023-12-29,364,1,365,1,True,2.0,2.0,3.0,Friday,HP - Wales North West Coast,House,4.0,6.0,4.0,True,False,Other settlement,Gwynedd,Wales,Coed Ystumgwern,North Wales,20.0,Super Income Maximisation,True,True,True,False,True,False,False,False,True,True,False,False,False,False,False,True,True,True,True,False,True,False,False,False,False,True,False,True,False,True,False,False,True,False,False,False,True,True,False,False,True,False,True,False,False,False,False,False,31.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, pointbiserialr
from sklearn.preprocessing import LabelEncoder
import itertools

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = np.sum((confusion_matrix - np.outer(confusion_matrix.sum(axis=1), confusion_matrix.sum(axis=0)) / confusion_matrix.sum())**2 / (np.outer(confusion_matrix.sum(axis=1), confusion_matrix.sum(axis=0)) / confusion_matrix.sum()))
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n * (min(confusion_matrix.shape)-1)))

def calculate_correlations(df, variables):
    df = df.copy()
    # Fill or drop NaN/Inf values
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    
    # Encode categorical columns
    label_encoders = {}
    for col in variables:
        if df[col].dtype == 'object':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            label_encoders[col] = le

    corr_matrix = pd.DataFrame(index=variables, columns=variables)

    # Calculate correlations
    for var1, var2 in itertools.combinations(variables, 2):
        if np.issubdtype(df[var1].dtype, np.number) and np.issubdtype(df[var2].dtype, np.number):
            # Both are numerical
            corr, _ = pearsonr(df[var1], df[var2])
            corr_matrix.loc[var1, var2] = corr
            corr_matrix.loc[var2, var1] = corr
        elif df[var1].dtype == 'object' and df[var2].dtype == 'object':
            # Both are categorical
            corr = cramers_v(df[var1], df[var2])
            corr_matrix.loc[var1, var2] = corr
            corr_matrix.loc[var2, var1] = corr
        else:
            # One numerical, one categorical
            if df[var1].dtype == 'object':
                categorical_var = var1
                numerical_var = var2
            else:
                categorical_var = var2
                numerical_var = var1

            if len(df[categorical_var].unique()) == 2:
                corr, _ = pointbiserialr(df[categorical_var], df[numerical_var])
                corr_matrix.loc[var1, var2] = corr
                corr_matrix.loc[var2, var1] = corr
            else:
                corr_matrix.loc[var1, var2] = np.nan
                corr_matrix.loc[var2, var1] = np.nan

    # Fill diagonal with 1s (self-correlation)
    np.fill_diagonal(corr_matrix.values, 1)

    # Display the correlation matrix
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(corr_matrix)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

    return corr_matrix

# Use




In [11]:
variables = ['Churn_Year_1', 'Available_Days', 'LagPeriod','PicsAtLive','AllowsPets',
'NumberOfPetsAllowed',
'Bathrooms',
'Bedrooms',
'ChangeOverDay',
'PricingArea',
'PropertyType',
'Rating',
'Sleeps',
'SykesTicks',
'isHoneyPot',
'isTestProperty',
'LocationType',
'County',
'Country',
'Location',
'WorkingRegion',
'CommissionRate',
'RevenueManagementOption',
'hasBroadband',
'hasDishwasher',
'hasEnclosedGarden',
'hasGames',
'hasGarden',
'hasHotTub',
'hasIndoorPool',
'hasLivestock',
'hasOffRoadParking',
'hasOpenFire',
'hasPoolTable',
'hasSauna',
'hasSpaOrGym',
'hasSpaJacuzzi',
'hasSwimmingPool',
'hasWashingMachine',
'isCoastal',
'hasCountryside',
'hasFishing',
'isIsolated',
'hasNationalPark',
'hasNationalTrust',
'isNearPub',
'isNearShop',
'isNearCanal',
'isNearCycling',
'isNearLake',
'isNearRailway',
'isNearRiver',
'isNearWalks',
'isNew',
'hasNoAnimals',
'hasShortBreaksAllowed',
'isChildFriendly',
'hasCotAvailable',
'isGroundFloor',
'hasLastMinuteBreakAllowed',
'isSmokeFree',
'isSmoking',
'hasCharacter',
'isFamily',
'isFarm',
'isInGroup',
'isLogCabin',
'isLuxury',
'isRomantic',
'isThatched',
'isUnusual'
             
]
correlation_matrix = calculate_correlations(base, variables)
# print(correlation_matrix)

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:32: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Churn_Year_1,Available_Days,LagPeriod,PicsAtLive,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,PricingArea,PropertyType,Rating,Sleeps,SykesTicks,isHoneyPot,isTestProperty,LocationType,County,Country,Location,WorkingRegion,CommissionRate,RevenueManagementOption,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual
Churn_Year_1,1,-0.306417,-0.032256,0.037636,0.007086,0.025558,-0.002582,0.026938,-0.016816,-0.020844,0.002003,-0.089819,0.04033,-0.089819,-0.032483,NaN,-0.017016,0.015588,-0.00613,0.005938,-0.035463,-0.070848,0.0073,0.034485,-0.010246,-0.01028,-0.016951,-0.037436,0.001856,-0.013722,0.047027,-0.041001,-0.074513,0.000695,-0.035081,0.008501,0.00235,0.023367,0.009304,-0.011796,-0.020744,0.089811,0.01049,-0.032922,0.023694,-0.016292,0.025622,0.018893,-0.000032,-0.008717,-0.119827,0.047842,0.049233,0.017495,-0.000044,0.035175,0.004063,0.062223,0.025023,0.047299,-0.005836,0.021447,-0.082504,0.027312,0.00435,-0.010099,0.007799,0.028399,-0.042767,0.018893,0.039559
Available_Days,-0.306417,1,-0.281249,-0.032435,0.047407,0.017687,-0.07028,-0.079062,0.018113,0.004159,-0.013403,-0.009269,-0.080936,-0.009269,-0.005698,NaN,-0.004326,0.018877,0.044947,0.007102,0.034917,0.178585,-0.121784,0.011851,-0.051919,0.03019,-0.050489,0.067146,-0.047919,0.034579,0.034335,0.057516,-0.000512,-0.075978,-0.030466,-0.033188,-0.048394,-0.016668,0.029162,-0.052004,0.05531,-0.030215,0.019192,0.038451,-0.007421,-0.026734,-0.088808,-0.002715,-0.004025,0.032976,0.03144,-0.041897,0.053729,-0.123569,-0.046719,0.007747,-0.056209,-0.109898,0.033938,-0.044985,0.068893,-0.010644,0.042503,-0.035775,-0.002683,0.024085,-0.005617,-0.106206,0.070747,0.011751,-0.012684
LagPeriod,-0.032256,-0.281249,1,-0.007959,-0.015487,0.011281,0.035692,0.05027,0.008642,-0.128269,0.05016,0.046536,0.035591,0.046536,0.072089,NaN,-0.105673,-0.102648,-0.02448,0.014502,-0.135485,-0.08314,0.058331,0.069165,0.048974,-0.047923,0.065947,0.000997,0.102969,0.026731,-0.024402,-0.000576,-0.006623,0.050957,0.082345,0.033443,-0.026478,0.091867,-0.03439,0.095882,0.010547,0.010727,-0.022784,-0.027376,-0.012415,-0.131968,0.026073,-0.001702,0.027596,-0.038456,-0.029774,-0.073914,-0.006525,-0.017202,0.029743,-0.068444,-0.004681,0.116569,0.061987,-0.077336,0.015688,-0.027202,0.04967,0.046045,0.021584,-0.025719,0.126738,0.040225,-0.00955,0.009134,0.012172
PicsAtLive,0.037636,-0.032435,-0.007959,1,0.02876,0.090822,0.45875,0.522442,-0.029502,-0.049525,0.13841,0.241162,0.502681,0.241162,-0.014619,NaN,-0.009599,-0.021495,0.018298,0.009622,-0.006546,-0.054713,-0.03751,0.080725,0.297541,0.140648,0.214411,0.130084,0.130011,-0.027458,0.014248,0.002957,0.201671,0.121222,0.06809,0.121177,0.036277,0.019267,0.249331,-0.029599,0.024261,0.004481,0.010005,0.062777,0.026475,-0.004958,-0.050657,-0.008124,-0.012403,0.000161,-0.007642,0.056171,0.010577,-0.011315,-0.030075,-0.107617,0.086754,0.109207,-0.127258,-0.115633,0.022358,-0.008216,0.056608,0.229851,0.01461,-0.028537,-0.080719,0.153829,-0.230891,0.002601,0.018431
AllowsPets,0.007086,0.047407,-0.015487,0.02876,1,0.819066,0.019036,0.108126,-0.048212,-0.016433,0.101315,-0.050974,0.101947,-0.050974,-0.040474,NaN,-0.063138,0.07988,0.009304,-0.003799,0.015257,0.025666,0.024884,0.031996,0.008645,0.179143,0.073351,0.105265,0.047386,0.007576,-0.053439,0.031282,0.103957,0.021591,-0.008357,-0.019919,0.041398,0.001254,0.074963,-0.004118,0.066672,-0.075212,0.012545,-0.054767,0.019679,-0.015162,-0.019074,-0.0122

In [13]:
base1 = base.dropna(subset=['Churn_Year_1'])
base1

,PropertyID,PropertyReference,PropertyLiveDate,DateToldWithdrawingLocal,Difference,AvailableFromDate,AvailableToDate,Available_Days,Not_Available_Days,Total_Days,...,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual,PicsAtLive,ChurnFlag,Churn_Year_1,Churn_Year_2,Churn_Year_1&2
0,223990,1128768,2023-02-28,NaT,0,2023-03-03,NaT,362,3,365,...,False,False,False,False,False,31.0,0.0,0.0,0.0,0.0
1,225256,1129878,2023-08-25,NaT,0,2023-08-25,NaT,365,0,365,...,False,False,False,False,False,17.0,0.0,0.0,0.0,0.0
2,221376,1126413,2023-02-10,2023-10-23,255,2023-03-31,2023-10-30,214,151,365,...,False,False,False,False,False,25.0,1.0,1.0,0.0,1.0
3,279394,1152055,2024-06-14,NaT,0,2024-06-24,NaT,355,10,365,...,False,False,False,False,False,24.0,0.0,0.0,0.0,0.0
4,210746,1119529,2022-12-17,2023-04-13,117,2022-12-18,2023-12-29,364,1,365,...,False,False,False,False,False,31.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8171,284049,1155772,2024-05-09,NaT,0,2024-05-10,NaT,364,1,365,...,False,False,False,False,False,30.0,0.0,0.0,0.0,0.0
8172,277099,1150231,2024-02-29,NaT,0,2024-03-01,NaT,364,1,365,...,False,False,False,False,True,30.0,0.0,0.0,0.0,0.0
8173,234957,1138337,2023-09-11,NaT,0,2023-09-11,NaT,365,0,365,...,False,False,False,False,False,22.0,0.0,0.0,0.0,0.0
8174,273044,1147631,2024-03-13,NaT,0,2024-03-13,NaT,365,0,365,...,False,False,False,False,False,50.0,0.0,0.0,0.0,0.0


In [14]:
PropertyBedroom2 = base1[base1['Bedrooms'] == 2]
bedroom2 = calculate_correlations(PropertyBedroom2, variables)

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:16: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:32: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

C:\Users\DeepankarSrigyan\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Churn_Year_1,Available_Days,LagPeriod,PicsAtLive,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,PricingArea,PropertyType,Rating,Sleeps,SykesTicks,isHoneyPot,isTestProperty,LocationType,County,Country,Location,WorkingRegion,CommissionRate,RevenueManagementOption,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual
Churn_Year_1,1,-0.3116,-0.039053,0.067375,0.020367,0.0383,0.017241,NaN,-0.023842,-0.00765,0.008632,-0.088397,-0.05325,-0.088397,-0.095029,NaN,-0.020439,0.018708,-0.064388,0.038232,-0.034843,-0.05636,-0.005194,-0.031757,-0.00979,-0.031054,-0.013419,-0.056167,-0.049505,-0.011022,0.059695,-0.079799,-0.072236,0.027972,-0.05063,NaN,-0.003144,0.042774,0.022551,0.001864,-0.061468,0.149403,-0.059591,-0.066474,0.044869,-0.069079,0.017942,0.039626,-0.004693,0.013153,-0.176695,0.082434,0.06822,0.056232,-0.022145,0.01035,-0.030826,0.049084,0.071371,0.036263,-0.039626,0.027972,-0.074853,0.000418,-0.081541,-0.046176,0.041707,0.037983,-0.104674,0.039626,0.062978
Available_Days,-0.3116,1,-0.232698,-0.019577,0.094065,0.049291,-0.055629,NaN,0.009355,0.028303,-0.01643,-0.00183,-0.010697,-0.00183,-0.02588,NaN,0.008747,0.054005,0.016345,0.006663,0.075956,0.191485,-0.131405,0.079915,-0.018145,0.109267,0.024163,0.099802,-0.039789,0.056683,0.046133,0.050279,0.032242,0.05095,-0.010549,NaN,-0.083093,-0.027532,0.098605,-0.128626,0.11716,-0.128438,0.071223,0.030917,-0.065724,0.046828,-0.075848,0.040978,0.02292,0.011559,0.018259,-0.018496,0.038237,-0.146258,-0.087379,0.053289,0.03669,-0.101804,-0.002122,-0.057354,0.081389,-0.046011,0.038191,-0.047525,0.047457,0.030154,-0.013242,-0.149738,0.031267,0.069544,-0.031597
LagPeriod,-0.039053,-0.232698,1,-0.028063,-0.07312,-0.041316,-0.021896,NaN,0.017894,-0.184596,0.029928,0.00744,-0.069007,0.00744,0.133325,NaN,-0.131764,-0.176465,-0.001077,0.006229,-0.185652,-0.122891,0.026588,0.129178,0.079687,-0.069907,0.072696,-0.0104,0.06511,0.033377,-0.019467,0.005733,-0.055747,-0.024028,0.12136,NaN,-0.039098,0.106021,-0.089287,0.161817,0.00156,0.060971,-0.035704,0.025591,-0.007679,-0.203119,0.014483,-0.036741,0.02713,-0.079246,0.000014,-0.079219,0.010329,-0.008041,0.104071,-0.084297,0.011664,0.206011,0.086806,-0.102942,0.037642,-0.035473,0.033736,0.043465,0.049958,-0.059844,0.13725,0.067449,0.012781,-0.028185,-0.008863
PicsAtLive,0.067375,-0.019577,-0.028063,1,-0.014687,0.043188,0.106877,NaN,-0.076476,-0.03216,0.000219,0.152445,-0.018978,0.152445,-0.050602,NaN,0.066897,0.03272,-0.021549,-0.012185,0.009636,-0.015337,-0.018147,0.05629,0.1571,0.042976,0.132566,0.103339,0.106035,-0.074667,-0.055017,-0.090221,0.086683,0.052914,-0.000996,NaN,0.050573,0.008064,0.118595,-0.06645,0.054381,0.057439,0.009104,0.044113,-0.038158,-0.046863,-0.073886,-0.078659,-0.008898,0.004166,-0.021833,0.063916,0.015138,0.017812,-0.023947,-0.061084,-0.029205,-0.039775,-0.080941,-0.114556,-0.010151,0.015638,-0.024104,-0.023545,-0.08276,-0.03145,-0.058541,0.133788,-0.079606,0.024553,-0.007518
AllowsPets,0.020367,0.094065,-0.07312,-0.014687,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.067243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011459,-0.04581,0.111671,0.029185,0.10906,0.035809,-0.001302,-0.033043,0.04887,0.097063,-0.008922,0.004577,NaN,0.042018,-0.013954,0.056807,-0.022272,0.140072,-0.081306,0.047035,-0.023091,0.019386,-0.037106,-0.063792,-0.09629,-0.030422,-0.053341,-0.098978,-0.026729,0.091502,0.01174,-0.928019,0.046557,-0.023106,0.022877,0.057152,0.001191,0.01264,-0.008922,0.02

In [15]:
PropertyBedroom3 = base1[base1['Bedrooms'] == 3]
bedroom3 = calculate_correlations(PropertyBedroom3, variables)

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:16: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:32: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

C:\Users\DeepankarSrigyan\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Churn_Year_1,Available_Days,LagPeriod,PicsAtLive,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,PricingArea,PropertyType,Rating,Sleeps,SykesTicks,isHoneyPot,isTestProperty,LocationType,County,Country,Location,WorkingRegion,CommissionRate,RevenueManagementOption,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual
Churn_Year_1,1,-0.303629,-0.01287,-0.026325,-0.022742,-0.004406,0.032695,NaN,-0.071235,-0.058647,-0.047189,-0.107021,0.07823,-0.107021,0.057245,NaN,-0.028597,-0.056657,0.08384,0.002264,-0.015372,-0.085502,0.042732,0.179106,-0.12468,-0.044085,-0.038888,-0.063557,0.069598,-0.026978,0.02551,-0.002076,-0.108438,-0.036224,-0.019838,NaN,-0.060633,0.018299,-0.045739,0.123265,-0.000343,0.062759,0.010193,0.028975,-0.0381,-0.107683,-0.089312,-0.060633,-0.078913,-0.033953,-0.062646,-0.022754,-0.02551,0.040037,0.049126,0.052573,0.018534,0.143078,0.0921,0.078998,-0.046289,NaN,-0.04282,0.048365,0.046289,0.064005,-0.0,0.001903,-0.002579,-0.019838,-0.051423
Available_Days,-0.303629,1,-0.29315,0.038714,0.021704,0.026013,0.005359,NaN,0.078137,-0.03234,0.047418,-0.01481,0.035029,-0.01481,0.005227,NaN,-0.005758,0.074566,0.001548,-0.024635,0.007121,0.174429,-0.078286,-0.032966,0.012073,-0.000411,0.029294,0.043975,0.04592,0.04872,0.044239,0.066983,-0.066536,-0.019616,-0.044507,NaN,0.0409,-0.008515,-0.006145,-0.032419,0.04878,0.077278,0.062665,0.081265,0.083886,-0.033959,-0.136909,-0.049465,0.039358,0.074074,0.112516,-0.080977,0.044423,-0.119485,-0.022542,-0.009405,-0.145832,-0.15981,0.046025,-0.030799,0.142493,NaN,0.068948,0.056169,0.029991,0.00142,0.080427,-0.054077,0.04005,-0.00275,-0.025128
LagPeriod,-0.01287,-0.29315,1,-0.051249,-0.004076,-0.010836,-0.043939,NaN,-0.002725,-0.160626,0.007353,-0.105184,-0.042271,-0.105184,-0.006235,NaN,-0.034606,-0.182157,-0.115632,0.070712,-0.1861,-0.082792,0.159473,0.037087,-0.049952,-0.077411,0.007479,-0.038209,-0.086879,-0.009594,-0.060398,-0.102364,-0.007772,-0.027918,-0.026479,NaN,0.019118,0.075339,0.033056,0.041387,-0.064357,0.035372,-0.004452,-0.022141,-0.014588,-0.057673,0.092324,0.029916,0.074357,-0.002654,-0.06605,-0.031868,-0.040917,-0.019893,-0.00375,-0.071692,-0.006207,0.065603,0.057259,-0.054229,-0.002053,NaN,0.056138,-0.023999,-0.041788,0.067323,0.060816,-0.036745,-0.025876,0.093911,0.058915
PicsAtLive,-0.026325,0.038714,-0.051249,1,-0.088766,-0.079617,0.225194,NaN,0.01848,-0.003587,-0.048108,0.157621,0.098799,0.157621,0.027336,NaN,0.061052,-0.053352,-0.015479,0.105745,0.066025,-0.058296,-0.034407,0.063007,0.202477,0.08076,0.13296,0.074237,0.105899,-0.027154,0.047026,0.083247,0.075316,0.092995,0.077957,NaN,-0.046658,-0.007188,0.148478,-0.045916,0.008258,-0.005947,-0.08337,0.072287,0.06409,0.022656,-0.048632,-0.038839,-0.031752,-0.05891,-0.012453,0.016545,0.051517,-0.028452,0.062123,-0.063559,0.027133,0.009218,-0.171917,-0.062874,-0.014217,NaN,0.06782,-0.023292,0.00036,0.073983,-0.096233,0.044599,-0.055214,0.011445,-0.051733
AllowsPets,-0.022742,0.021704,-0.004076,-0.088766,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.072996,NaN,-0.029612,NaN,NaN,NaN,NaN,NaN,NaN,-0.003256,-0.108495,0.215328,0.07249,0.094004,-0.001094,0.009317,-0.113449,-0.016895,0.089661,0.01355,-0.069477,NaN,-0.012972,-0.000171,0.048229,-0.042994,0.078165,-0.104549,-0.015749,-0.012933,0.012226,0.057366,-0.010978,-0.012972,-0.036693,-0.016488,-0.037085,-0.029884,0.010625,0.01355,-0.875712,0.071713,0.046743,-0.023649,-0.060518,0.022641,-0.032695,NaN,-0.029518

In [16]:
PropertyBedroom4 = base1[base1['Bedrooms'] == 4]
bedroom4 = calculate_correlations(PropertyBedroom4, variables)

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:16: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:32: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

C:\Users\DeepankarSrigyan\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Churn_Year_1,Available_Days,LagPeriod,PicsAtLive,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,PricingArea,PropertyType,Rating,Sleeps,SykesTicks,isHoneyPot,isTestProperty,LocationType,County,Country,Location,WorkingRegion,CommissionRate,RevenueManagementOption,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual
Churn_Year_1,1,-0.260134,-0.113128,0.037664,0.089792,0.112152,-0.117776,NaN,-0.032317,0.067368,0.042191,-0.124236,-0.0,-0.124236,-0.052753,NaN,-0.061435,0.14748,0.090181,0.017509,0.107128,-0.061559,-0.052202,-0.005103,-0.03135,0.045218,0.007362,-0.053072,0.012423,NaN,0.044947,-0.114806,-0.079869,-0.065475,NaN,0.063806,NaN,-0.04576,-0.150389,-0.068181,-0.018464,-0.016933,0.117028,0.121716,0.006265,0.17166,0.119401,0.063806,0.045813,0.121716,-0.02616,0.105409,-0.078446,NaN,-0.042643,0.151526,-0.092498,-0.06989,-0.061274,0.214449,-0.044947,NaN,-0.134597,-0.165653,0.063806,-0.065696,0.078446,-0.003163,0.078446,0.063806,0.063806
Available_Days,-0.260134,1,-0.383862,0.182886,0.077583,0.0703,-0.043248,NaN,0.028907,-0.021833,-0.082407,0.078642,-0.016495,0.078642,-0.052918,NaN,0.038581,-0.139178,0.03329,-0.048424,-0.020824,0.028154,-0.106683,-0.065754,-0.029639,-0.001492,-0.066546,0.185198,0.032106,NaN,0.06687,0.043179,0.100558,-0.191703,NaN,-0.107791,NaN,0.019057,0.208276,-0.037147,-0.079494,0.145823,0.001423,-0.105795,0.008666,-0.101262,-0.154002,0.090187,-0.035312,-0.025352,-0.013357,0.00483,0.201513,NaN,-0.160188,-0.053121,0.055705,-0.049195,0.066994,-0.107664,0.199525,NaN,0.138085,0.191427,-0.00406,0.019336,0.068127,-0.04559,0.002053,-0.104828,-0.040811
LagPeriod,-0.113128,-0.383862,1,-0.103176,-0.109933,-0.099198,0.166355,NaN,0.141719,-0.065364,-0.032276,0.164132,-0.001298,0.164132,0.044153,NaN,-0.091151,-0.086539,-0.021047,-0.022869,-0.112184,0.067352,-0.068351,0.014931,0.000427,-0.08713,0.060176,0.054119,0.075043,NaN,-0.052551,0.163156,0.022974,0.109996,NaN,-0.074602,NaN,0.148725,-0.100169,-0.020854,0.156269,-0.147161,-0.027426,-0.040134,-0.042027,-0.041743,-0.0649,-0.060715,-0.044197,0.021361,-0.139682,-0.021302,-0.013564,NaN,0.117127,-0.075874,0.054666,0.03898,0.000692,-0.054648,-0.128418,NaN,-0.118735,-0.10179,-0.074602,-0.082089,-0.071801,0.142602,0.079011,-0.0503,0.201397
PicsAtLive,0.037664,0.182886,-0.103176,1,-0.067574,-0.112322,0.136311,NaN,-0.051171,0.04036,0.047406,0.243468,0.030138,0.243468,-0.129104,NaN,0.057717,0.093113,0.049049,0.065708,-0.022871,-0.060072,-0.057445,0.172365,0.074176,0.015369,-0.099086,0.122236,0.043425,NaN,-0.101099,0.023004,0.172825,-0.17383,NaN,0.024116,NaN,0.107509,0.169292,-0.070466,0.147891,-0.119104,0.064729,0.018594,0.108803,-0.215284,-0.164347,0.035292,0.082127,-0.011862,-0.00742,0.134096,0.20393,NaN,0.080267,0.038773,0.201785,0.018469,-0.086513,0.013898,0.211313,NaN,-0.026614,0.130977,-0.059702,-0.030922,0.025069,-0.139138,-0.02989,-0.137932,0.09117
AllowsPets,0.089792,0.077583,-0.109933,-0.067574,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.012919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.105599,-0.02591,0.148287,0.147386,0.015496,-0.014298,NaN,-0.1197,-0.098666,0.098271,0.081888,NaN,-0.169926,NaN,0.004002,-0.102841,-0.003101,-0.119067,-0.028768,-0.059641,-0.182567,0.186143,0.087419,0.129423,0.089846,0.068609,-0.040984,0.031045,-0.064534,-0.004002,NaN,-0.917465,0.197341,-0.071243,0.037083,0.05397,0.289367,-0.06329,NaN,0.230251,0.003658,0.089846,-0.007098,-0.208915,0.094883,0.004002,0.089846,0.089846
NumberOfPetsAllowed,

In [17]:
PropertyBedroomGT4 = base1[base1['Bedrooms'] > 4]
bedroomGT4 = calculate_correlations(PropertyBedroomGT4, variables)

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\793191077.py:16: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\DeepankarSrigyan\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Churn_Year_1,Available_Days,LagPeriod,PicsAtLive,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,PricingArea,PropertyType,Rating,Sleeps,SykesTicks,isHoneyPot,isTestProperty,LocationType,County,Country,Location,WorkingRegion,CommissionRate,RevenueManagementOption,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual
Churn_Year_1,1,-0.368215,-0.024793,0.071311,-0.057225,0.043497,-0.078907,0.064198,0.03702,-0.021728,0.247661,0.014056,0.172211,0.014056,-0.13609,NaN,-0.038235,0.111613,0.011801,-0.124924,0.053463,-0.144681,-0.010463,NaN,0.037807,0.051496,-0.080163,-0.127081,0.022463,0.046569,-0.070917,-0.009625,-0.062237,0.067753,0.06619,-0.01299,0.081479,0.081479,0.112679,-0.098881,0.008266,0.160992,0.025215,-0.217583,0.032645,0.117371,0.130451,0.046569,0.132258,-0.132258,0.062237,0.10156,0.03809,NaN,0.029435,0.169385,0.194488,0.155128,-0.016518,0.028283,NaN,NaN,-0.050509,-0.170999,0.046569,0.132258,NaN,0.002054,0.046569,NaN,-0.070917
Available_Days,-0.368215,1,-0.266443,-0.122995,0.090985,-0.022407,-0.057193,-0.144451,-0.047736,-0.166871,-0.082163,-0.077477,-0.185766,-0.077477,0.382601,NaN,0.089885,-0.086717,0.050929,0.083434,0.100499,0.378372,-0.053745,NaN,-0.164404,0.009134,-0.19862,-0.035049,-0.097539,0.010934,0.04668,-0.019847,-0.102474,-0.215784,-0.19032,-0.020204,-0.156567,0.018091,-0.065058,0.162671,-0.042316,-0.052678,-0.095597,0.195576,0.028487,-0.027563,0.071302,0.010934,-0.006239,-0.03607,0.014406,-0.183279,0.04929,NaN,-0.073226,0.073385,-0.326821,-0.12873,0.018504,0.030704,NaN,NaN,0.038893,0.1014,-0.015805,-0.098587,NaN,-0.142804,0.105411,NaN,0.027447
LagPeriod,-0.024793,-0.266443,1,-0.067783,0.03473,0.022161,-0.004982,0.032202,0.109585,0.019743,-0.118183,0.074626,0.009604,0.074626,-0.017497,NaN,0.01985,0.159097,-0.150493,-0.099816,-0.176716,-0.202183,-0.110089,NaN,-0.167784,-0.041392,0.172985,0.03988,0.23375,0.189188,-0.041577,0.008201,0.185969,0.207577,0.132355,0.162089,-0.104609,0.122305,-0.129935,0.016354,-0.021571,-0.183296,0.099626,-0.175318,-0.136264,-0.111261,-0.050932,0.189188,-0.194976,-0.086754,0.03463,-0.111319,-0.011082,NaN,-0.057775,-0.083579,-0.135354,-0.163027,-0.117193,-0.158394,NaN,NaN,0.105929,-0.031063,-0.065393,-0.026157,NaN,0.056011,-0.065393,NaN,0.19094
PicsAtLive,0.071311,-0.122995,-0.067783,1,-0.124077,-0.007234,0.154609,0.038535,-0.06002,-0.092606,0.181058,0.191268,0.040668,0.191268,-0.07406,NaN,-0.087456,-0.117748,0.134477,-0.120544,-0.06616,0.024135,-0.046226,NaN,0.116358,-0.052833,0.106286,0.055881,-0.03127,-0.043505,0.251177,-0.152438,0.054846,0.057648,0.298803,0.091463,0.010928,-0.034336,0.010298,-0.065786,0.044864,0.058493,0.072127,0.218714,0.061878,0.032016,0.014321,-0.043505,-0.034238,0.026063,-0.063154,0.028565,-0.051075,NaN,0.133414,-0.341239,-0.027106,0.100995,0.129356,-0.20656,NaN,NaN,0.12408,0.059273,0.278883,0.066587,NaN,0.265606,-0.115147,NaN,0.188504
AllowsPets,-0.057225,0.090985,0.03473,-0.124077,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071617,0.073685,0.04479,0.092264,0.09145,0.061932,0.108359,0.144732,-0.03342,-0.09092,0.088027,-0.062594,0.108359,0.108359,0.007605,0.138851,0.124992,-0.160346,0.081916,0.038434,-0.150202,-0.202881,0.028882,0.061932,-0.141341,-0.062594,-0.099822,-0.136287,-0.125766,NaN,-0.882692,-0.003515,-0.176745,-0.02115,0.041423,-0.030225,NaN,NaN,-0.127747,0.114383,0.061932,-0.141341,NaN,-0.194327,0.061932,NaN,0.108359
NumberOfPetsAllowed,0.043497

## RandomForestRegressor model, explaining the model's predictions using SHAP (SHapley Additive exPlanations)

## Cross Validation approach

In [19]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
import shap
import matplotlib.pyplot as plt



# Define independent and dependent variables
dependent_variable = 'Churn_Year_1'  # Replace with your actual target variable name
independent_variables = ['Available_Days', 'LagPeriod', 'PicsAtLive']

# Create X (features) and y (target)
X = base1[independent_variables]
y = base1[dependent_variable]

# Create and train a model (using Random Forest as an example)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Initialize SHAP explainer and calculate SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Ensure the plots are rendered correctly
plt.switch_backend('agg')  # Use 'agg' backend for rendering to files

# Summary plot
shap.summary_plot(shap_values, X)
plt.savefig('summary_plot.png')  # Save the figure

# Bar plot of feature importances
shap.summary_plot(shap_values, X, plot_type="bar")
plt.savefig('bar_plot.png')  # Save the figure

# Print feature importances
feature_importance = pd.DataFrame(list(zip(X.columns, np.abs(shap_values).mean(axis=0))),
                                  columns=['feature', 'importance'])
feature_importance = feature_importance.sort_values('importance', ascending=False)
print(feature_importance)

# Add predictions to the DataFrame
predictions = model.predict(X)
results = base1.copy()
results['Predicted_Churn_Probability'] = predictions
# results
finalResult=results[['PropertyReference','Available_Days','LagPeriod','PicsAtLive','Churn_Year_1','Predicted_Churn_Probability']]
finalResult

C:\Users\DeepankarSrigyan\AppData\Roaming\Python\Python311\site-packages\shap\plots\_beeswarm.py:952: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\DeepankarSrigyan\AppData\Roaming\Python\Python311\site-packages\shap\plots\_beeswarm.py:952: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



          feature  importance
0  Available_Days    0.193090
1       LagPeriod    0.120845
2      PicsAtLive    0.033016


,PropertyReference,Available_Days,LagPeriod,PicsAtLive,Churn_Year_1,Predicted_Churn_Probability
0,1128768,362,3,31.0,0.0,0.054864
1,1129878,365,0,17.0,0.0,0.135072
2,1126413,214,49,25.0,1.0,1.000000
3,1152055,355,10,24.0,0.0,0.000000
4,1119529,364,1,31.0,1.0,0.302381
...,...,...,...,...,...,...
8171,1155772,364,1,30.0,0.0,0.000000
8172,1150231,364,1,30.0,0.0,0.000000
8173,1138337,365,0,22.0,0.0,0.077239
8174,1147631,365,0,50.0,0.0,0.000000


In [20]:
from sklearn.metrics import f1_score, roc_curve

def find_optimal_threshold(data, churn_probability):
    # Extract the actual labels and the predicted probabilities
    actual = data['Churn_Year_1']
    probabilities = data[churn_probability]
    
    # Calculate the ROC curve to get thresholds
    fpr, tpr, thresholds = roc_curve(actual, probabilities)
    
    # Calculate F1 scores for each threshold
    f1_scores = []
    for thresh in thresholds:
        predictions = (probabilities >= thresh).astype(int)
        score = f1_score(actual, predictions)
        f1_scores.append(score)
    
    # Find the threshold that maximized the F1 score
    max_f1_index = np.argmax(f1_scores)  # Index of the maximum F1 score
    optimal_threshold = thresholds[max_f1_index]
    max_f1_score = f1_scores[max_f1_index]
    
    print(f'Optimal Threshold: {optimal_threshold}')
    print(f'Maximum F1 Score: {max_f1_score:.4f}')
    
    # Optionally, return the optimal threshold
    return optimal_threshold, max_f1_score


# find_optimal_threshold(data, 'churn_probability')

find_optimal_threshold(finalResult, 'Predicted_Churn_Probability')


Optimal Threshold: 0.39361904761904754
Maximum F1 Score: 0.7899


(0.39361904761904754, 0.7899461400359067)

In [21]:
def confusion_chart(data, threshold, churn_probability, actual_churn):
    import pandas as pd
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve, auc, precision_recall_curve, f1_score
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    # Calculate predicted churn based on the given threshold
    data['predicted_churn'] = (data[churn_probability] >= threshold).astype(int)
    
    # Calculate precision, recall, and F1 score
    precision = precision_score(data[actual_churn], data['predicted_churn'])
    recall = recall_score(data[actual_churn], data['predicted_churn'])
    f1 = f1_score(data[actual_churn], data['predicted_churn'])
    
    print(f'Precision: {precision:.2%}')
    print(f'Recall: {recall:.2%}')
    print(f'F1 Score: {f1:.2%}')
    
    # Calculate confusion matrix
    cm = confusion_matrix(data[actual_churn], data['predicted_churn'])
    tn, fp, fn, tp = cm.ravel()
    
    # Display confusion matrix values
    print("\nConfusion Matrix:")
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    # Create confusion matrix heatmap with custom annotations
    plt.figure(figsize=(8, 6))
    ax = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
    
    # Add custom text annotations for TP, FP, TN, FN
    labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
    counts = [tn, fp, fn, tp]
    percentages = cm / cm.sum() * 100
    label_counts = [f'{label}\n{count}\n({percent:.2f}%)' for label, count, percent in zip(labels, counts, percentages.flatten())]
    
    for (j, i), label in np.ndenumerate(cm):
        ax.text(i + 0.5, j + 0.5, label_counts[j * 2 + i], ha='center', va='center', color='black')
    
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Create bar chart to show predicted vs actual
    metrics = ['True Positives', 'False Positives', 'True Negatives', 'False Negatives']
    values = [tp, fp, tn, fn]
    
    plt.figure(figsize=(8, 6))
    plt.bar(metrics, values, color=['green', 'red', 'blue', 'orange'])
    plt.title('Predicted vs Actual Cases')
    plt.xlabel('Metrics')
    plt.ylabel('Number of Cases')
    plt.show()

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(data[actual_churn], data[churn_probability])
    roc_auc = auc(fpr, tpr)
    
    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()
    
    # Calculate Precision-Recall curve
    precision_vals, recall_vals, _ = precision_recall_curve(data[actual_churn], data[churn_probability])
    pr_auc = auc(recall_vals, precision_vals)
    
    # Plot Precision-Recall curve
    plt.figure(figsize=(8, 6))
    plt.plot(recall_vals, precision_vals, color='blue', lw=2, label=f'Precision-Recall curve (area = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.show()
    
    # Calculate Gain and Lift chart
    data_sorted = data.sort_values(by=churn_probability, ascending=False)
    data_sorted['cumulative_churns'] = data_sorted[actual_churn].cumsum()
    data_sorted['cumulative_data'] = np.arange(1, len(data_sorted) + 1)
    data_sorted['gain'] = data_sorted['cumulative_churns'] / data_sorted[actual_churn].sum()
    data_sorted['lift'] = data_sorted['gain'] / (data_sorted['cumulative_data'] / len(data_sorted))
    
    # Plot Gain chart
    plt.figure(figsize=(8, 6))
    plt.plot(data_sorted['cumulative_data'] / len(data_sorted), data_sorted['gain'], color='green', lw=2, label='Gain')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
    plt.xlabel('Cumulative % of Data')
    plt.ylabel('Cumulative % of Churns')
    plt.title('Gain Chart')
    plt.legend(loc="lower right")
    plt.show()
    
    # Plot Lift chart
    plt.figure(figsize=(8, 6))
    plt.plot(data_sorted['cumulative_data'] / len(data_sorted), data_sorted['lift'], color='purple', lw=2, label='Lift')
    plt.xlabel('Cumulative % of Data')
    plt.ylabel('Lift')
    plt.title('Lift Chart')
    plt.legend(loc="upper right")
    plt.show()
 
    
    # Display table of metrics
    metrics_data = {
        "Metric": ["Precision", "Recall", "ROC AUC", "PR AUC"],
        "Value": [precision, recall, roc_auc, pr_auc]
    }
    metrics_df = pd.DataFrame(metrics_data)
    print("\nMetrics Table:")
    print(metrics_df)




In [22]:
confusion_chart(finalResult, 0.39, 'Predicted_Churn_Probability', 'Churn_Year_1')

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:47: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:58: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:74: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:87: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



Precision: 91.10%
Recall: 69.68%
F1 Score: 78.96%

Confusion Matrix:
True Negatives: 6341
False Positives: 86
False Negatives: 383
True Positives: 880

Metrics Table:
      Metric     Value
0  Precision  0.910973
1     Recall  0.696754
2    ROC AUC  0.944992
3     PR AUC  0.864489


C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:104: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\DeepankarSrigyan\AppData\Local\Temp\ipykernel_11268\1817947100.py:113: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

